In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()

%matplotlib notebook
pd.set_option('display.max_columns',500)

In [355]:
nse = pd.read_csv('citi_denue.csv')

In [356]:
df = pd.read_excel('Income_BASE JAM2.xlsx',dtype={'codigo_postal':'object'})

In [362]:
data = pd.merge(df,nse,how='left',left_on='ID',right_on='ID').drop(['codigo_postal_x','codigo_postal_y'],1)

In [364]:
data['Income_Range'] = data['Income_Range'].astype('object')

In [368]:
data.rename(columns={'ID':'customer_id'},inplace=True)

In [372]:
tmp_prof = pd.read_excel('demographic_info.xlsx',sheet_name='task3')
tmp_educ = pd.read_excel('demographic_info.xlsx',sheet_name='task4')
tmp_sex  = pd.read_excel('demographic_info.xlsx',sheet_name='task5')

In [425]:
preprocessed_data  = data.merge(tmp_prof,how='left',left_on='customer_id',right_on='ID').drop(['ID','V_PROFESSION_CD_x','V_PROFESSION_CD_y'],1)  \
                         .merge(tmp_educ,how='left',left_on='customer_id',right_on='ID').drop(['ID','N_COLL_SCHOLARSHIP_LEVEL_x'],1)  \
                         .merge(tmp_sex,how='left',left_on='customer_id',right_on='ID').drop(['ID','V_SEX'],1)

In [427]:
preprocessed_data.loc[(preprocessed_data['EDAD']>=18) & (preprocessed_data['EDAD']<=24), 'age_grp'] = '18-24'
preprocessed_data.loc[(preprocessed_data['EDAD']>=25) & (preprocessed_data['EDAD']<=54), 'age_grp'] = '25-34'
preprocessed_data.loc[(preprocessed_data['EDAD']>=25) & (preprocessed_data['EDAD']<=54), 'age_grp'] = '35-44'
preprocessed_data.loc[(preprocessed_data['EDAD']>=25) & (preprocessed_data['EDAD']<=54), 'age_grp'] = '45-54'
preprocessed_data.loc[(preprocessed_data['EDAD']>=55) & (preprocessed_data['EDAD']<=64), 'age_grp'] = '55-64'
preprocessed_data.loc[(preprocessed_data['EDAD']>=65), 'age_grp'] = '65+'

In [429]:
preprocessed_data.columns = ['income','age','cust_id','nse','profession','education','sex','age_grp']

In [430]:
preprocessed_data.loc[preprocessed_data['nse'].isnull(), 'nse'] = 'no_info'

In [443]:
dumm_sex = pd.get_dummies(preprocessed_data['sex']).drop('MEN',1)
dumm_nse = pd.get_dummies(preprocessed_data['nse']).drop('no_info',1)
dumm_age = pd.get_dummies(preprocessed_data['age_grp'])

dumm_profession = pd.get_dummies(preprocessed_data['profession']).drop('NO INFO',1)
dumm_education  = pd.get_dummies(preprocessed_data['education']).drop('WITHOUT SCHOLARSHIP',1)

In [451]:
preprocessed_data#.drop('age',1,inplace=True)
preprocessed_data.drop('age_grp',1,inplace=True)

In [452]:
df_final = pd.concat([preprocessed_data,dumm_sex,dumm_profession,dumm_education,dumm_age],1).drop(['nse','profession','education','sex'],1)

In [453]:
df_final.set_index('cust_id',inplace=True)

In [455]:
X = df_final.drop('income',1)
y = df_final['income'].astype('int')

In [549]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [550]:
rf = RandomForestClassifier(n_estimators=150)

In [551]:
rf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [552]:
print(rf.score(x_train, y_train))
print(rf.score(x_test, y_test))

0.3595651306602833
0.3616476611589481


In [553]:
results = pd.DataFrame()
results['y_real'] = y_test
results['y_pred'] = rf.predict(x_test)

In [554]:
import numpy as np
results['error'] = np.abs(results['y_real'] - results['y_pred'])

In [555]:
results.reset_index()[['cust_id','error']].groupby('error').count()

,cust_id
error,
0,1554
1,2052
2,483
3,165
4,38
5,5


In [556]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',                                                                 ascending=False)

In [539]:
# probs = pd.DataFrame(rf.predict_proba(x_test),columns=[1,2,3,4,5,6,7])
# probs['weighted_1'] = probs[1]*10.761082
# probs['weighted_2'] = probs[2]*3.137568
# probs['weighted_3'] = probs[3]*3.152762
# probs['weighted_4'] = probs[4]*4.961205
# probs['weighted_5'] = probs[5]*40.120448
# probs['weighted_6'] = probs[6]*31.688053
# probs['weighted_7'] = probs[7]*76.186170

In [559]:
from scipy.stats import kendalltau

tau, p_value = kendalltau(list(results['y_real']),list(results['y_pred']))

In [560]:
p_value

8.288353360642698e-51

In [561]:
tau

0.20370906684591636